# Automatic Differentiation

## Installing Autograd

You need to install Autograd by typing ```conda install --channel conda-forge autograd``` in your console. We will test it on toy function.

In [ ]:
import autograd.numpy as np
from autograd import elementwise_grad
import matplotlib.pyplot as plt

In [ ]:
def f(x):
    return np.abs(x)

gradient_f = elementwise_grad(f)

In [ ]:
x = np.linspace(-3,3,1000)
plt.figure(figsize=(15,6))
plt.plot(f(x), label='$f$')
plt.plot(gradient_f(x), label="$f'$")
plt.legend(fontsize=20)
plt.show()

## Application to Neural Network training: Geographical Origins of Music

In [ ]:
import autograd.numpy as np
from autograd import grad
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

### Loading the data

The goal of this exercise is to train a two-layer feed-forward neural network in a regression problem. The data consists of music features and location. The goal is to retrieve the spatial origin of a music piece using the music features.

In [ ]:
# Load the Geographical Origins of Music dataset
data = pd.read_table('Geographical Original of Music/default_plus_chromatic_features_1059_tracks.txt', sep=',', header=None)
data.rename(columns={116:'Latitude', 117:'Longitude'}, inplace=True)

We split the dataset in two : a train dataset with 80% of the data, and a test data set with the remaining 20% of the data.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-2], data.iloc[:,-2:], test_size=0.20, random_state=42)

Let us plot the geographical repartition in the train and test datasets.

In [ ]:
fig = plt.figure(figsize=(17, 13))

# Train data
ax = fig.add_subplot(1, 2, 1, projection=ccrs.Robinson())

ax.stock_img()
ax.coastlines()

ax.set_title('Train data')
ax.scatter(y_train['Longitude'], y_train['Latitude'], c='r', transform=ccrs.Geodetic())

# Test data
ax2 = fig.add_subplot(1, 2, 2, projection=ccrs.Robinson())

ax2.stock_img()
ax2.coastlines()

ax2.set_title('Test data')
ax2.scatter(y_test['Longitude'], y_test['Latitude'], c='r', transform=ccrs.Geodetic())

plt.show()

### Prediction of Longitude/Latitude with a 2-layer neural network

We implement the two-layer neural network.

In [ ]:
# Size of the neural network layers
size_features = X_train.shape[1]
size_hidden_layer = # TODO: Try different values for the number of neurons in the hidden layer
size_prediction = 2 # We want to predict the latitude and the longitude

In [ ]:
# Compute the min and max of the Latitudes and Longitudes in the train dataset
# This will be useful for rescaling the output of the neural network
min_latitude = y_train['Latitude'].min()
max_latitude = y_train['Latitude'].max()
min_longitude = y_train['Longitude'].min()
max_longitude = y_train['Longitude'].max()

In [ ]:
# Define the targets and inputs of the neural network
targets = np.array(y_train)
inputs = np.array(X_train.T)

In [ ]:
def network(inputs, A1, A2, b1, b2, sgd=False):
    '''Return the predicted value for features x with a 2-layer neural network of parameters A1 and A2 and biases b1 and b2.'''
    assert inputs.shape[0] == A1.shape[1]
    assert A2.shape == (2, A1.shape[0])
    if sgd: # If only one data is to be computed
        [latitude, longitude] = # TODO: return the objective value when SGD is run
    else: # If the network has to compute several outputs 
        [latitude, longitude] = # TODO: return the objective value when GD is run
    latitude = min_latitude + 0.5*(latitude+1)*(max_latitude - min_latitude)
    longitude = min_longitude + 0.5*(longitude+1)*(max_longitude - min_longitude)
    return np.array([latitude, longitude]).T

In [ ]:
def loss(y_pred, y_true):
    '''Return the arc-cosine distance on the Earth between y_pred and y_true.'''
    latitude_pred, longitude_pred = y_pred*np.pi/180.
    latitude_true, longitude_true = y_true*np.pi/180.
    return # TODO: search for the arc-cosine distance formula on Wikipedia

In [ ]:
def objective(weights):
    '''Compute the objective to be minimized by the network using a gradient descent approach.'''
    A1, A2, b1, b2 = weights
    pred = network(inputs, A1, A2, b1, b2)
    return # TODO: return the loss

In [ ]:
def objective_sgd(weights):
    '''Compute the objective to be minimized by the network using a stochastic gradient descent approach.'''
    A1, A2, b1, b2 = weights
    i = np.random.randint(inputs.shape[1])
    pred = network(inputs[:,i], A1, A2, b1, b2, sgd=True)
    return # TODO: return the loss

In [ ]:
def objective_test(weights):
    '''Compute the objective for the test dataset.'''
    A1, A2, b1, b2 = weights
    pred = network(X_test.T, A1, A2, b1, b2)
    return # TODO: return the loss

#### Gradient Descent

We fist use a Gradient Descent approach to minimize the loss function

In [ ]:
np.random.seed(0)

epochs = 40 # Number of epochs to be ran by the Gradient Descent algorithm
learning_rate = 0.00005

# Initialize the weights and biases of the network
A1 = np.random.randn(size_hidden_layer, size_features)
A2 = np.random.randn(size_prediction, size_hidden_layer)
b1 = np.random.randn(size_hidden_layer)
b2 = np.random.randn(size_prediction)

# Compute the gradient of the objective
grad_loss = # TODO: use Autograd's function `grad`

train_loss_history = []
test_loss_history = []
print('Iteration', '|', 'Train loss', '|', 'Test loss')
for t in range(epochs):
    train_loss_history.append(objective([A1,A2,b1,b2]))
    test_loss_history.append(objective_test([A1,A2,b1,b2]))
    print(t, '        | ', int(train_loss_history[-1]), 'km   | ', int(test_loss_history[-1]), 'km')
    
    # Compute the gradients
    grad_A1, grad_A2, grad_b1, grad_b2 = # TODO: use the preci-omputed gradient `grad_loss`
    
    # Update the weights and biases
    A1 = # TODO: use Gradient Descent update formula
    A2 = # TODO: use Gradient Descent update formula
    b1 = # TODO: use Gradient Descent update formula
    b2 = # TODO: use Gradient Descent update formula

# Save the weights and biases
A1_GD, A2_GD, b1_GD, b2_GD = A1, A2, b1, b2

In [ ]:
min_objective = min([min(train_loss_history), min(test_loss_history)])
plt.figure(figsize=(16,5))
plt.semilogy(train_loss_history-min_objective, lw=4, label='Train loss')
plt.semilogy(test_loss_history-min_objective, lw=4, label='Test loss')
plt.xlabel('Iterations', fontsize=25)
plt.ylabel('Loss', fontsize=25)
plt.legend(fontsize=25)
plt.show()

#### Stochastic Gradient Descent

As we observe that the convergence of the Gradient Descent approach is slow, we also try a Stochastic Gradient Descent Approach. Since the objective is non convex, a stochastic approach will more likely avoid being stuck in local minima.
We use a decaying learning rate in $1/\sqrt{t}$ where $t$ is the number of iterations, and a Polyak-Ruppert averaging. This will always be the case in the rest of this notebook.

Since the Stochastic Gradient Descent approach seems to converge way faster than the Gradient Descent approach in this case, we do not use the sae number of epochs : the SGD algorithm will use less data in this very application.

In [ ]:
np.random.seed(0)

epochs = 10 # Number of passes over the data
learning_rate = 0.00005

# Initialize the weights and biases of the network
A1 = np.random.randn(size_hidden_layer, size_features)
A2 = np.random.randn(size_prediction, size_hidden_layer)
b1 = np.random.randn(size_hidden_layer)
b2 = np.random.randn(size_prediction)

# Initialize the averaged weights and biases of the network
A1_moy = np.copy(A1)
A2_moy = np.copy(A2)
b1_moy = np.copy(b1)
b2_moy = np.copy(b2)

# Compute the gradient of the objective
grad_loss = # TODO

train_loss_history = []
test_loss_history = []

print('Number of passes', '|', 'Train loss', '|', 'Test loss')
for t in range(epochs*X_train.shape[0]):
    if t % X_train.shape[0] == 0:
        train_loss_history.append(objective([A1_moy,A2_moy,b1_moy,b2_moy]))
        test_loss_history.append(objective_test([A1_moy,A2_moy,b1_moy,b2_moy]))
        print(int(t/X_train.shape[0]), '             | ', int(train_loss_history[-1]), 'km   | ', int(test_loss_history[-1]), 'km')
    
    # Compute the gradients
    grad_A1, grad_A2, grad_b1, grad_b2 = # TODO
    
    # Update the weights and biases
    A1 = # TODO: use SGD update formula, with strep_size decaying as 1/sqrt(t)
    A2 = # TODO: use SGD update formula, with strep_size decaying as 1/sqrt(t)
    b1 = # TODO: use SGD update formula, with strep_size decaying as 1/sqrt(t)
    b2 = # TODO: use SGD update formula, with strep_size decaying as 1/sqrt(t)
    
    # Compute the averaged weights and biases (Polyak-Rupert)
    A1_moy = (t*A1_moy + A1)/(t+1)
    A2_moy = (t*A2_moy + A2)/(t+1)
    b1_moy = (t*b1_moy + b1)/(t+1)
    b2_moy = (t*b2_moy + b2)/(t+1)
    
    
# Save the weights and biases
A1_SGD, A2_SGD, b1_SGD, b2_SGD = A1_moy, A2_moy, b1_moy, b2_moy

In [ ]:
min_objective = min([min(train_loss_history), min(test_loss_history)])
plt.figure(figsize=(16,5))
plt.semilogy(train_loss_history-min_objective, lw=4, label='Train loss')
plt.semilogy(test_loss_history-min_objective, lw=4, label='Test loss')
plt.xlabel('Iterations', fontsize=25)
plt.ylabel('Loss', fontsize=25)
plt.legend(fontsize=25)
plt.show()

#### Results

Let us plot some random test data, and the predicted values for the GD approach and the SGD approach

We can see that in both cases, the network fails to predict correctly the origin of a music piece. The train loss, as well as the test loss, are approximately of 4000km, which is clearly not precise enough. In particular, the network seems to always predict points at the center of the map (Europe/North Africa/Middle East), since this is a good strategy for musics coming from this area (predicting Europe instead of North Africa is cheap in terms of the arc-cosine loss), and not too costly for musics coming from South America/East Asia/Oceania.

In [ ]:
i = np.random.randint(X_test.shape[0])

pred_SGD = network(X_test.iloc[i,:].T, A1_SGD, A2_SGD, b1_SGD, b2_SGD, True)
pred_GD = network(X_test.iloc[i,:].T, A1_GD, A2_GD, b1_GD, b2_GD, True)

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

ax.stock_img()
ax.coastlines()

ax.scatter(y_test['Longitude'].iloc[i], y_test['Latitude'].iloc[i], c='r', s=100, transform=ccrs.Geodetic(), label='Real data')
ax.scatter([pred_SGD[0]], [pred_SGD[1]], c='k', transform=ccrs.Geodetic(), s=100, label='Predicted value with SGD')
ax.scatter([pred_GD[0]], [pred_GD[1]], c='b', transform=ccrs.Geodetic(), s=100, label='Predicted value with GD')

ax.set_title('Test data N°'+str(i))

plt.legend()
plt.show()

### Predicting a heatmap

We now wish to predict a heatmap instead of a fixed location.

In [ ]:
# Load the music data
data = pd.read_table('Geographical Original of Music/default_plus_chromatic_features_1059_tracks.txt', sep=',', header=None)
data.drop(data.columns[-2:], axis=1, inplace=True)
data = np.array(data)

In [ ]:
# Load the geographical maps
maps = np.array(pd.read_table('heatmap.txt', sep='   ', header=None, engine='python'))

In [ ]:
# Concatenate music data and geographical maps
data = np.concatenate([data, maps], axis=1)

In [ ]:
# Split the dataset in two : train dataset (80% of the data) and a test dataset (the 20% remaining)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[:,:-400], data[:,-400:], test_size=0.20, random_state=42)

In [ ]:
# Plot a random data
i = np.random.randint(y_train.shape[0])
plt.imshow(y_train[i].reshape(20,20))
plt.show()

#### Model

In [ ]:
# Size of the neural network layers
size_features = X_train.shape[1]
size_hidden_layer = # TODO: Try different values for the number of neurons in the hidden layer
size_prediction = 400 # We want to predict a 20x20 image

In [ ]:
# Define the targets and inputs of the neural network
targets = np.array(y_train)
inputs = np.array(X_train.T)

In [ ]:
def softmax(x):
    '''Return the softmax of vector x.'''
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
def network(inputs, A1, A2, b1, b2, sgd=False):
    '''Return the predicted value for features x with a 2-layer neural network of weights A1 and A2 and biases b1 and b2.'''
    assert inputs.shape[0] == A1.shape[1]
    assert A2.shape == (400, A1.shape[0])
    if sgd: # If there is only one data to be predicted
        return # TODO: return the prediction of the network
    else: # If there is several data to be passed to the network
        return # TODO: return the prediction of the network

#### Gradient Descent with a KL loss

In [ ]:
def entropy(p, q):
    '''Return the entropy of p wrt q.'''
    tol = 0.001
    assert p.shape[0] == q.shape[0]
    assert np.prod(q>0) == 1
    return -np.sum(p*np.log(q+tol))
    
def objective_KL(weights):
    '''Compute the objective to be minimized with a KL (entropy) loss and a gradient descent approach.'''
    A1, A2, b1, b2 = weights
    pred = network(inputs, A1, A2, b1, b2)
    return # TODO: return the KL loss

def objective_KL_sgd(weights):
    '''Compute the objective to be minimized with a KL (entropy) loss and a stochastic gradient descent approach.'''
    A1, A2, b1, b2 = weights
    i = np.random.randint(inputs.shape[1])
    pred = network(inputs[:,i], A1, A2, b1, b2, True)
    return # TODO: return the KL loss

def objective_KL_test(weights):
    '''Compute the objective with a KL (entropy) loss for the test dataset.'''
    A1, A2, b1, b2 = weights
    pred = network(X_test.T, A1, A2, b1, b2)
    return # TODO: return the KL loss

In [ ]:
np.random.seed(0)

epochs = 50 # Number of epochs to be run
learning_rate = 2.

# Initialize the network's weigths and biases
A1 = 0.01*np.random.randn(size_hidden_layer, size_features)
A2 = 0.01*np.random.randn(size_prediction, size_hidden_layer)
b1 = 0.01*np.random.randn(size_hidden_layer)
b2 = 0.01*np.random.randn(size_prediction)

In [ ]:
# Compute the gradient of the objective
grad_loss = # TODO: use Autograd

train_loss_history = []
test_loss_history = []

print('Iteration', '|', 'Train loss', '         |', 'Test loss')
for t in range(epochs):
    train_loss_history.append(objective_KL([A1,A2,b1,b2]))
    test_loss_history.append(objective_KL_test([A1,A2,b1,b2]))
    print(t, '        | ', train_loss_history[-1], ' | ', test_loss_history[-1])
    
    # Compute the gradients of the parameters
    grad_A1, grad_A2, grad_b1, grad_b2 = # TODO
    
    # Run the descent step
    A1 = # TODO
    A2 = # TODO
    b1 = # TODO
    b2 = # TODO

# Save the parameters
A1_KL_GD, A2_KL_GD, b1_KL_GD, b2_KL_GD = A1, A2, b1, b2

In [ ]:
min_objective = min([min(train_loss_history), min(test_loss_history)])
plt.figure(figsize=(16,5))
plt.semilogy(train_loss_history-min_objective, lw=4, label='Train loss')
plt.semilogy(test_loss_history-min_objective, lw=4, label='Test loss')
plt.xlabel('Iterations', fontsize=25)
plt.ylabel('Loss', fontsize=25)
plt.legend(fontsize=25)
plt.show()

#### Stochastic Gradient Descent with a KL loss

In [ ]:
np.random.seed(0)

epochs = 50 # Number of passes over the data
learning_rate = 2.

# Initialize the network's parameters
A1 = 0.01*np.random.randn(size_hidden_layer, size_features)
A2 = 0.01*np.random.randn(size_prediction, size_hidden_layer)
b1 = 0.01*np.random.randn(size_hidden_layer)
b2 = 0.01*np.random.randn(size_prediction)

# Initialize the averaged parameters
A1_moy = np.copy(A1)
A2_moy = np.copy(A2)
b1_moy = np.copy(b1)
b2_moy = np.copy(b2)

In [ ]:
# Compute the gradient of the SGD objective
grad_loss = # TODO: use Autograd

train_loss_history = []
test_loss_history = []

print('Number of passes', '|', 'Train loss', '         |', 'Test loss')
for t in range(epochs*X_train.shape[0]):
    if t % X_train.shape[0] == 0:
        train_loss_history.append(objective_KL([A1_moy,A2_moy,b1_moy,b2_moy]))
        test_loss_history.append(objective_KL_test([A1_moy,A2_moy,b1_moy,b2_moy]))
        print(int(t/X_train.shape[0]), '             | ', train_loss_history[-1], ' | ', test_loss_history[-1])
    
    # Compute the gradients of the parameters
    grad_A1, grad_A2, grad_b1, grad_b2 = # TODO
    
    # Run the descent step
    A1 = # TODO
    A2 = # TODO
    b1 = # TODO
    b2 = # TODO
    
    # Run the averaging step
    A1_moy = (t*A1_moy + A1)/(t+1)
    A2_moy = (t*A2_moy + A2)/(t+1)
    b1_moy = (t*b1_moy + b1)/(t+1)
    b2_moy = (t*b2_moy + b2)/(t+1)
    
# Save the parameters
A1_KL_SGD, A2_KL_SGD, b1_KL_SGD, b2_KL_SGD = A1_moy, A2_moy, b1_moy, b2_moy

In [ ]:
min_objective = min([min(train_loss_history), min(test_loss_history)])
plt.figure(figsize=(16,5))
plt.semilogy(train_loss_history-min_objective, lw=4, label='Train loss')
plt.semilogy(test_loss_history-min_objective, lw=4, label='Test loss')
plt.xlabel('Iterations', fontsize=25)
plt.ylabel('Loss', fontsize=25)
plt.legend(fontsize=25)
plt.show()

#### Results

Let us plot some random test data, and the predicted values for the GD approach and the SGD approach (for the KL/entropy loss). As the test losses show, the Stochastic Gradient Approach leads to slightly better predictions than the GD approach.

In [ ]:
i = np.random.randint(X_test.shape[0])

pred_GD = network(X_test[i], A1_KL_GD, A2_KL_GD, b1_KL_GD, b2_KL_GD, True)
pred_SGD = network(X_test[i], A1_KL_SGD, A2_KL_SGD, b1_KL_SGD, b2_KL_SGD, True)

fig = plt.figure(figsize=(15, 10))

ax = fig.add_subplot(1, 3, 1)
ax.imshow(y_test[i].reshape(20,20))
ax.set_title('Real data')

ax2 = fig.add_subplot(1, 3, 2)
ax2.imshow(pred_GD.reshape(20,20))
ax2.set_title('Prediction with GD')

ax3 = fig.add_subplot(1, 3, 3)
ax3.imshow(pred_SGD.reshape(20,20))
ax3.set_title('Prediction with SGD')

plt.show()